In [ ]:
import cv2

dataset link= 'https://www.kaggle.com/c/dogs-vs-cats'

In [ ]:
import numpy as np
import os
from random import shuffle

TRAIN_DIR ='C:/Users/DELL/Desktop/python/tensor/cnn/dog_vs_cat/train'
TEST_DIR ='C:/Users/DELL/Desktop/python/tensor/cnn/dog_vs_cat/test'
IMG_SIZE=50
LR=1e-3 #0.001

MODEL_NAME='dogsvscats-{}-{}.model'.format(LR,'2conv-basic')

In [ ]:
def label_img(img):
    word_label=img.split('.')[-3]
    if word_label=='cat':
        return [1,0]
    elif word_label=='dog':
        return [0,1]

In [ ]:
def create_train_data():
    training_data=[]
    for img in os.listdir(TRAIN_DIR):
        label=label_img(img)
        path=(TRAIN_DIR+'/'+img)
        img=cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data

In [ ]:
def process_test_data():
    testing_data=[]
    for img in os.listdir(TEST_DIR):
        path=(TRAIN_DIR+'/'+img)
        img_num=img.split('.')[0]
        img=cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img),img_num])
    np.save('test_data.npy',testing_data)
    return testing_data

In [ ]:
train_data=create_train_data()
# already train
#train_data=np.load('train_data.npy')

In [ ]:
len(train_data)

In [ ]:
train_data

In [ ]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print("model loaded!")

In [ ]:
X=np.array([i[0] for i in train_data])
#X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
X=X/255.0
X=X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y=np.array([i[1] for i in train_data])

In [ ]:
len(X)

In [ ]:
len(y)

In [ ]:
X

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(50,50,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(2, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from tensorflow import keras
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Dogvscat")

In [ ]:
tuner_search.search(X,y,epochs=10, validation_split=0.1)

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(X,y, epochs=20, validation_split=0.1, initial_epoch=10)

In [ ]:
model.save('2convo-catvsdog.model')

In [ ]:
CATEGORIES = ['Cat', 'Dog']


def image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    new_arr = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    new_arr = np.array(new_arr)
    new_arr = new_arr.reshape(-1,IMG_SIZE,IMG_SIZE, 1)
    return new_arr

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
image_plt = load_img('test/4.jpg', target_size=(224, 224))
plt.imshow(image_plt)

In [ ]:
y_pred=model.predict([image('test/4.jpg')])

In [ ]:
y_pred

In [ ]:
CATEGORIES[y_pred.argmax()]

In [ ]:
image_plt = load_img('test/2.jpg', target_size=(224, 224))
plt.imshow(image_plt)

In [ ]:
y_pred=model.predict([image('test/2.jpg')])
CATEGORIES[y_pred.argmax()]

In [ ]:
image_plt = load_img('test/45.jpg', target_size=(224, 224))
plt.imshow(image_plt)

In [ ]:
y_pred=model.predict([image('test/45.jpg')])
CATEGORIES[y_pred.argmax()]